In [1]:
import os
import json

CONFIG_DIR = "../.human_eval_config"
RESDULTS_DIR = "../results"

In [2]:
import zipfile
import datetime

os.makedirs(".new_backup", exist_ok=True)
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Backing up the Configs and Results
with zipfile.ZipFile(os.path.join(".new_backup", f"config_{current_time}"), 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(CONFIG_DIR):
        for file in files:
            zipf.write(os.path.join(root, file))

with zipfile.ZipFile(os.path.join(".new_backup", f"results_{current_time}"), 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(RESDULTS_DIR):
        for file in files:
            zipf.write(os.path.join(root, file))

In [3]:
current_time

'2024-02-18_07-55-01'

In [4]:
with open(os.path.join(CONFIG_DIR, "config.json"), "r") as f:
    config = json.load(f)

In [5]:
results_files = [f for f in os.listdir(RESDULTS_DIR) if f.endswith(".json")]

time_threshold = 180
workers = []
question_sets = []
completed_question_sets = []
incomplete_question_sets = []
fast_question_sets = []
for f in results_files:
    with open(os.path.join(RESDULTS_DIR, f)) as file:
        try:
            results = json.load(file)
            workers.append(results["worker_id"])
            question_sets.append(results["question_set_id"])
            if results["question_index"] == config["config"]["n_questions_per_worker"]:
                completed_question_sets.append(results["question_set_id"])
                time_taken = results["end_time"] - results["start_time"]
                if time_taken < time_threshold:
                    fast_question_sets.append((results["question_set_id"], f))
            else:
                incomplete_question_sets.append(results["question_set_id"])
        except Exception as e:
            print(f"Error in {f}: {e}")
        

Error in 14735989_9d13e20b-aa25-4aa3-bb84-ff10fcaf3383.json: Expecting value: line 1 column 1 (char 0)
Error in 14951301_c343bb68-7613-456b-894f-9b9782d338a4.json: Expecting value: line 1 column 1 (char 0)
Error in cw_14936765@hotmail.com_d43b7c1c-21c2-40e8-8219-458d308b3628.json: Expecting value: line 1 column 1 (char 0)
Error in 14815777_c23cfd95-5adc-4cbf-b137-1b3d3277fbfe.json: Expecting value: line 1 column 1 (char 0)
Error in 10031931_addab51b-72de-4070-86f3-001c824d5b17.json: Expecting value: line 1 column 1 (char 0)


In [6]:
print("Number of completed question sets:", len(set(completed_question_sets)))
print("Number of incomplete question sets:", len(set(incomplete_question_sets)))
print("Number of unique workers:", len(set(workers)))
print("Number of total workers:", len(workers))
print("Number of question sets:", len(question_sets))
print("Number of unique question sets:", len(set(question_sets)))
print("Number of fast question sets:", len(set(fast_question_sets)))

Number of completed question sets: 250
Number of incomplete question sets: 127
Number of unique workers: 376
Number of total workers: 449
Number of question sets: 449
Number of unique question sets: 250
Number of fast question sets: 1


In [7]:
fast_question_sets

[('397bfcff-acae-496a-9c0b-ede9f95381f8',
  'Ashish_397bfcff-acae-496a-9c0b-ede9f95381f8.json')]

In [8]:
# completed_question_sets - set([x[0] for x in fast_question_sets]) = correctly_completed
correctly_completed = [x for x in completed_question_sets if x not in set([x[0] for x in fast_question_sets])]

In [9]:
# completed_question_sets = correctly_completed
len(set(correctly_completed))

249

In [10]:
with open(os.path.join(CONFIG_DIR, "distribution.json"), "r") as f:
    distribution = json.load(f)

distribution

{'2a6660ae-99df-4c12-b28e-1a253e34369b': [['eb5f52c7-d80a-4415-b8bd-94afc211559a',
   {'orca2_7b': '52c6a78e-577c-4611-b17c-3023a051462b',
    'llama2_7b-chat': '1ffe2d89-0212-4ac6-abdf-2ec29e5ebce4'}],
  ['eb5f52c7-d80a-4415-b8bd-94afc211559a',
   {'orca2_7b': '96e2b34a-0c11-4db9-870b-52ffff9775e6',
    'mistral_7b-instruct': 'eed7c1f5-62a3-431f-8caa-70807fc03f0a'}],
  ['e3474732-8bf8-463d-8b57-b84fbeccf369',
   {'neural-chat_7b': 'e052438f-255e-49f2-9a70-54924fd29f21',
    'mistral_7b-instruct': 'e98d59d5-3bdb-469a-8566-4cd0c6191868'}],
  ['e3474732-8bf8-463d-8b57-b84fbeccf369',
   {'neural-chat_7b': 'a6d6c712-6234-4466-9385-c8ceaa1b9587',
    'zephyr_7b-beta': '435cdd4a-4120-468a-b703-de59646dc6da'}],
  ['516cea79-e5e1-4ea8-a733-a3dea2eb50c7',
   {'neural-chat_7b': '1b5ba5f0-8e7a-4512-8b11-5a0e058120be',
    'zephyr_7b-beta': 'e00a473b-8739-4ece-8c6c-8e28f0412ff6'}],
  ['516cea79-e5e1-4ea8-a733-a3dea2eb50c7',
   {'orca2_7b': '4f62a2a1-ade9-4125-854b-5f6dab52dba1',
    'vicuna_7b': '

In [11]:
len(distribution)

4

In [12]:
distribution_with_incpmlete = distribution.copy()
for k in distribution.keys():
    if k in set(completed_question_sets):
        distribution_with_incpmlete.pop(k, None)
len(distribution_with_incpmlete)

0

In [17]:
backup_check = input("Have you backed up the results and config files? (y/n) ")
if backup_check == "y":
    with open (os.path.join(CONFIG_DIR, "distribution.json"), "w") as f:
        json.dump(distribution_with_incpmlete, f, indent=4)
    updated_config = config.copy()
    updated_config["config"]["n_workers"] = len(distribution_with_incpmlete)
    with open (os.path.join(CONFIG_DIR, "config.json"), "w") as f:
        json.dump(updated_config, f, indent=4)
    os.remove(os.path.join(CONFIG_DIR, "worker_count.txt"))
else:
    print("Please backup the results and config files before proceeding.")

In [1]:
!cat ../../multi/.human_eval_config/config.json 

{
    "hv_method": "A/B Testing with Criterions",
    "config": {
        "n_workers": 4,
        "n_questions_per_worker": 10,
        "n_options": 2,
        "ability_to_tie": "Allow",
        "data_sources": [
            "peptalk_responses.json",
            "first_task_responses.json",
            "recommendation_responses.json",
            "all_task_completed_responses.json",
            "new_user_responses.json"
        ],
        "evenly_distributed": true,
        "show_captcha": true,
        "completion_code": "c9b2a7c5-995d-4df6-8c13-d7733ee7b319"
    },
    "time_created": "15/02/2024 17:22:03"
}

In [2]:
!cat ../../multi/.human_eval_config/worker_count.txt

4

In [3]:
!cat ../../prefer/.human_eval_config/config.json 

{
    "hv_method": "A/B Testing",
    "config": {
        "n_workers": 1,
        "n_questions_per_worker": 10,
        "n_options": 2,
        "ability_to_tie": "Allow",
        "data_sources": [
            "peptalk_responses.json",
            "first_task_responses.json",
            "recommendation_responses.json",
            "all_task_completed_responses.json",
            "new_user_responses.json"
        ],
        "evenly_distributed": true,
        "show_captcha": true,
        "completion_code": "7155bc11-bb03-4ca7-8281-2e06bb2ca212"
    },
    "time_created": "15/02/2024 17:19:56"
}

In [4]:
!cat ../../prefer/.human_eval_config/worker_count.txt

1

In [20]:
# rm -rf .human_eval_config/
# rm -rf results
# cp -R ../prefer/.human_eval_config/ .
# cp -R ../prefer/results/ .

# rm -rf ../prefer/.human_eval_config/
# cp -R .human_eval_config/ ../prefer/ 

In [21]:
# rm -rf .human_eval_config/
# rm -rf results
# cp -R ../multi/.human_eval_config/ .
# cp -R ../multi/results/ .

# rm -rf ../multi/.human_eval_config/
# cp -R .human_eval_config/ ../multi/ 

In [13]:
import pandas as pd

results_files = [f for f in os.listdir(RESDULTS_DIR) if f.endswith(".json")]

df = pd.DataFrame(columns=["worker_id", "question_set_id", "prompt_id", "response_a_model", "response_b_model", "overall_preference", "feedback", "time_taken", "total_time_taken"])
for f in results_files:
    with open(os.path.join(RESDULTS_DIR, f)) as file:
        try:
            result = json.load(file)
            total_time_taken = result["end_time"] - result["start_time"]
            worker_id = result["worker_id"]
            question_set_id = result["question_set_id"]
            for eval in result["evals"]:
                overall_preference = eval["result"]["overall"]
                feedback = eval["result"]["feedback"]
                time_taken = eval["time"] - result["start_time"]
                prompt_id = eval["question"][0]
                response_a_model, response_b_model = list(eval["question"][1].keys())
                new_row = pd.DataFrame({"worker_id": [worker_id], "question_set_id": [question_set_id], "prompt_id": [prompt_id], "response_a_model": [response_a_model], "response_b_model": [response_b_model], "overall_preference": [overall_preference], "feedback": [feedback], "time_taken": [time_taken], "total_time_taken": [total_time_taken]})
                df = pd.concat([df, new_row], ignore_index=True)
        except Exception as e:
            print(f"Error in {f}: {e}")

/tmp/ipykernel_1026236/616372585.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


Error in 14735989_9d13e20b-aa25-4aa3-bb84-ff10fcaf3383.json: Expecting value: line 1 column 1 (char 0)
Error in 14951301_c343bb68-7613-456b-894f-9b9782d338a4.json: Expecting value: line 1 column 1 (char 0)
Error in cw_14936765@hotmail.com_d43b7c1c-21c2-40e8-8219-458d308b3628.json: Expecting value: line 1 column 1 (char 0)
Error in 14815777_c23cfd95-5adc-4cbf-b137-1b3d3277fbfe.json: Expecting value: line 1 column 1 (char 0)
Error in 10031931_addab51b-72de-4070-86f3-001c824d5b17.json: Expecting value: line 1 column 1 (char 0)


In [14]:
df.to_csv("multi_results.csv", index=False)

In [15]:
# convert to a excel file with additional column with a dropdown ["Accept", "Reject", "Maybe"]
# in the worker_id if consective rows have the same value, then merge the cells in the excel file. do it for both worker_id and question_set_id, total_time_taken, dropdown columns

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
from openpyxl.styles import Font
from openpyxl.styles import Border, Side

wb = openpyxl.Workbook()
ws = wb.active

for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)

for col in ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try: 
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

for r in range(1, len(df)+2):
    if r == 1:
        ws.cell(row=r, column=10, value="Accept/Reject/Maybe")
        ws.cell(row=r, column=10).alignment = Alignment(horizontal='center')
        ws.cell(row=r, column=10).fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type = "solid")
        ws.cell(row=r, column=10).font = Font(bold=True)
    else:
        ws.cell(row=r, column=10).alignment = Alignment(horizontal='center')
        ws.cell(row=r, column=10).border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

start_row = 2
end_row = 2
while end_row <= len(df) + 1:
    while end_row <= len(df) + 1 and ws.cell(row=end_row, column=1).value == ws.cell(row=end_row+1, column=1).value:
        end_row += 1
    if start_row != end_row:
        print(start_row, end_row)
        ws.merge_cells(start_row=start_row ,start_column=1, end_row=end_row, end_column=1)
        ws.merge_cells(start_row=start_row ,start_column=2, end_row=end_row, end_column=2)
        ws.merge_cells(start_row=start_row ,start_column=9, end_row=end_row, end_column=9)
        ws.merge_cells(start_row=start_row ,start_column=10, end_row=end_row, end_column=10)
        start_row = end_row + 1
        end_row = start_row
    else:
        start_row += 1
        end_row += 1

wb.save("multi_results.xlsx")


2 5
6 14
15 24
26 35
37 46
47 51
52 61
64 67
68 77
78 87
88 97
98 107
110 119
120 127
128 137
138 147
148 157
158 167
168 177
178 183
184 193
195 204
205 207
208 217
218 227
229 238
239 248
249 258
259 268
269 278
279 285
286 295
297 306
307 310
311 320
321 330
331 340
342 344
345 354
355 356
357 366
367 376
377 386
387 393
394 397
398 407
408 417
418 427
428 429
430 439
440 449
450 459
460 464
465 474
475 484
485 494
495 504
505 506
507 516
517 521
522 523
524 533
534 543
545 550
552 561
563 572
573 582
584 588
589 597
599 606
607 616
617 626
628 637
638 647
648 653
654 663
664 673
674 683
684 693
697 706
709 718
719 728
729 738
739 748
749 758
760 763
764 773
774 783
784 793
794 803
804 813
815 824
825 827
828 837
838 847
848 853
854 863
865 867
868 877
878 887
888 897
900 909
910 919
920 929
930 939
940 941
942 951
952 959
960 969
970 974
975 984
985 994
995 1004
1005 1014
1015 1024
1025 1034
1035 1037
1041 1045
1046 1047
1048 1057
1058 1061
1062 1071
1072 1081
1082 1091
1092 1101
1

In [18]:
class Color:
    def __init__(self, cake):
        self.cake = cake


class Fruit:
    def __init__(self, name:str, color: Color):
        self.name = name
        self.color =  color



kwargs = {"name": "apple", "color": {"cake": "red_velvent"}}

fruit = Fruit(**kwargs)

In [19]:
fruit.color

{'cake': 'red_velvent'}

In [20]:
from dataclasses import dataclass

In [21]:
@dataclass
class Person:
    name: str
    age: int
    job: str

    def something(self):
        print(self.name)

In [22]:
chandra = Person(name="chandra", age=28, job="Ml engineer")

In [23]:
chandra.something()

chandra
